# Easy GPT-Q + LoRA in JAX ([github](https://github.com/davisyoshida/easy-lora-and-gptq))

[Davis Yoshida](https://github.com/davisyoshida/)

This notebook shows how to combine  two JAX tools/transforms I wrote: [Lorax](https://github.com/davisyoshida/lorax) and [JAX-GPTQ](https://github.com/davisyoshida/jax-gptq). I've been using the combination to run LLaMA finetunes on a single GPU.

They're both applicable to basically any JAX function, which conveniently includes many HuggingFace models!

The procedure is as follows:

1. Quantize the weights of the model we want to use
2. Use Lorax to transform the original model function `F(params, inputs)` to one that takes a tuple of the original params and the low rank LoRA params: `F_lora(param_tuple, inputs)`
3. Wrap `F_lora` in `use_quantized` transform so that it knows how to handle arguments which are int8 matrices with two parameters per byte.
4. Train the model, updating only the low rank params and leaving the larger 4-bit model weights frozen.

I'd love feedback on one or both of these tools so please let me know on their Githubs if you have any suggestions. JAX-GPTQ in particular is still in a really early state.

####XLA Runtime OOM Prevention

In [ ]:
import os

# Allocate 90% of the GPU memory to the XLA runtime
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".9"

# Disable preallocation of memory
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

# Use the platform allocator instead of the cuda allocator
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

### Setup

In [ ]:
!pip install git+https://github.com/davisyoshida/jax-gptq.git
!pip install jax-lorax
!pip install transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/davisyoshida/jax-gptq.git to /tmp/pip-req-build-k_jo2l0c
  Running command git clone --filter=blob:none --quiet https://github.com/davisyoshida/jax-gptq.git /tmp/pip-req-build-k_jo2l0c
  Resolved https://github.com/davisyoshida/jax-gptq.git to commit 8b8ff0fd23b4a7732f1c5dca98d7275045194d3c
  Preparing metadata (setup.py) ... done
  Created wheel for jax-gptq: filename=jax_gptq-0.0.1-py3-none-any.whl size=16385 sha256=a2859bad302537b7f25b2bee3f4c1b5bbbb271b30821e6db4b595b038197e9e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ck8sz67p/wheels/ff/5e/fb/dec939c953c916b7437c0ce0839617a79dc06e0a2fd85138a2
Successfully built jax-gptq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from functools import partial
import jax
import jax.numpy as jnp
import numpy as np
import optax
import torch

import transformers
from transformers import (
    CONFIG_MAPPING,
    FLAX_MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForCausalLM,
    HfArgumentParser,
    TrainingArguments,
    is_tensorboard_available,
)

from tqdm import trange

import lorax
import jax_gptq

gpu = jax.devices('gpu')[0]
cpu = jax.devices('cpu')[0]

## Toy Example

### Model/Data setup

First we'll define an MLP and make some parameters for it:

In [ ]:
from transformers import LongT5Config, FlaxT5ForConditionalGeneration
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer

from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the LongT5-XL model with its configuration
model_id = "google/long-t5-tglobal-xl"
config = LongT5Config.from_pretrained(model_id)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=nf4_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Initialize the model parameters using JAX's PRNG key
rng_key = jax.random.PRNGKey(0)
input_ids = jnp.array([[1, 2, 3, 4, 5]])
decoder_input_ids = jnp.array([[1, 2, 3, 4, 5]])
params = model.parameters()  # Returns an iterable over the parameters 
#params = model.named_parameters() # Returns an iterable over the parameters and their names

# Modify my_model to use the LongT5-XL model instead of the custom model defined earlier
def my_model(params, x):
    logits = model(input_ids=x, params=params, train=True).logits
    return jnp.mean(logits)

# Define a loss function for the LongT5-XL model
@jax.jit
def compute_loss(params, input_ids, decoder_input_ids, labels):
  logits = model(
      input_ids=input_ids, 
      decoder_input_ids=decoder_input_ids,   
      params=params, 
      train=True
  ).logits

# Transform the loss function to get the gradients
grad_fn = jax.value_and_grad(compute_loss)

# Define an optimizer to update the parameters using the gradients
optimizer = optax.adam(learning_rate=1e-3)

# Define a train step function which combines the loss function and optimizer update, does the forward and backward pass, and returns the updated parameters
@jax.jit
def train_step(params, x, y, optimizer):
    grads, loss = grad_fn(params, x, y)
    updates, optimizer_state = optimizer.update(grads, optimizer_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, loss, optimizer_state

# Define a batch generator function using get_batches() from stackoverflow.com
def generate_batch(batch_size, rng, DIM=512):
    # Generate a batch of input-output pairs
    X_batch = list(jax.random.normal(rng, (batch_size, DIM)))
    Y_batch = jax.random.randint(rng, (batch_size,), 0, 2, dtype=jnp.int32)

    return X_batch, Y_batch

# Initialize the optimizer state and the PRNG key
optimizer_state = optimizer.init(params)
rng = jax.random.PRNGKey(0)

# Train the model
num_steps = 50
batch_size = 4

for i in range(num_steps):
    # Generate a batch of input-output pairs
    x_batch, y_batch = generate_batch(batch_size, rng)
    
    # Update the parameters and optimizer state
    params, loss, optimizer_state = train_step(params, x_batch, y_batch, optimizer_state)
    
    # Print the loss every 10 steps
    if i % 10 == 0:
        print(f'Step {i}, Loss: {loss}')    


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 46>:46                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optax/_src/combine.py:50 in init_fn                      │
│                                                                                                  │
│    47   init_fns, update_fns = zip(*transforms)                                                  │
│    48                                                                                            │
│    49   def init_fn(params):                                                                     │
│ ❱  50 │   return tuple(fn(params) for fn in init_fns)                                            │
│    51                                                                                            │
│    52   def update_fn(updates, state, params=None, **extra_args):                                │
│    53 │   if len(update_fns) != len(state):                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optax/_src/combine.py:50 in <genexpr>                    │
│                                                                                                  │
│    47   init_fns, update_fns = zip(*transforms)                                                  │
│    48                                                                                            │
│    49   def init_fn(params):                                                                     │
│ ❱  50 │   return tuple(fn(params) for fn in init_fns)                                            │
│    51                                                                                            │
│    52   def update_fn(updates, state, params=None, **extra_args):                                │
│    53 │   if len(update_fns) != len(state):                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/optax/_src/transform.py:335 in init_fn                   │
│                                                                                                  │
│    332   mu_dtype = utils.canonicalize_dtype(mu_dtype)                                           │
│    333                                                                                           │
│    334   def init_fn(params):                                                                    │
│ ❱  335 │   mu = jax.tree_util.tree_map(  # First moment                                          │
│    336 │   │   lambda t: jnp.zeros_like(t, dtype=mu_dtype), params)                              │
│    337 │   nu = jax.tree_util.tree_map(jnp.zeros_like, params)  # Second moment                  │
│    338 │   return ScaleByAdamState(count=jnp.zeros([], jnp.int32), mu=mu, nu=nu)                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/jax/_src/tree_util.py:210 in tree_map                    │
│                                                                                                  │
│   207   """                                                                                      │
│   208   leaves, treedef = tree_flatten(tree, is_leaf)                                            │
│   209   all_leaves = [leaves] + [treedef.flatten_up_to(r) for r in rest]                         │
│ ❱ 210   return treedef.unflatten(f(*xs) for xs in zip(*all_leaves))                              │
│   211                                                      

GPT-Q needs input data for quantization. For an actual model we'd use real data but here we'll just make some random inputs.

In [ ]:
quant_data = [jax.random.normal(key, (batch_size, DIM)) for key in jax.random.split(data_key, 64)]

# We'll save an output for later comparison since the quantization process will delete the original params
original_output = my_model(params, quant_data[0])

### Run GPT-Q to get the quantized weights
That's all for the setup, we can now just run GPT-Q (without any changes to the original model code):

In [ ]:
# Note that this may free the buffers associated with some or all of the parameters and the data to save VRAM
# I'd also recommend you put the params on the CPU, since `quantize()` will move the params to th GPU when necessary
quantized_params = jax_gptq.quantize(my_model, params, quant_data)

The matrices have been quantized but the biases have been left alone:

In [ ]:
 print(f'W type: {type(quantized_params[0]["w"])}')
 print(f'B type: {type(quantized_params[0]["b"])}')

**Note**: The quantization procedure depends on the parameter being used in a matrix multiplication. Currently JAX-GPTQ supports general dot operations (including ones using tensors with any number of dimensions larger than 1), and convolutions with kernels of spatial size 1.

### Applying the quantized weights
We can now run the quantized model without any code changes. All that's necessary is using `jax_gptq.use_quantized` to transform the function so it knows how to handle `QuantizedMatrix` values.

In [ ]:
quantized_params = jax.device_put(quantized_params, gpu) # Move the params to the GPU

# Originally:
# my_model(params, inputs)
# After:
# jax_gptq(my_model)(params, inputs)
quant_output = jax_gptq.use_quantized(my_model)(quantized_params, quant_data[0])

print(f'Output of quantized network: {quant_output:.3e}')
print(f'Original output: {original_output:.3e}')

### Train with LoRA

Now that we've compressed our model to 4-bits (and change) per parameter, we can add full precision LoRA parameters for finetuning.

The one gotcha about combining the two is that Lorax doesn't know that QuantizedMatrix values are pytree leaves, so you need to give the Lorax functions an `is_leaf` predicate.

**Initialization:** The `init_lora` function expects a pytree describing which parameters should get LoRA parameters, which should be fully trained, and which should be left frozen. `lorax.simple_spec` is a helper function for making these specs.

In [ ]:
def is_leaf(x):
  return isinstance(x, jax_gptq.QuantizedMatrix)

lora_spec = lorax.simple_spec(
    params=quantized_params,
    decision_fn=lambda pytree_path, arr: 4, # Just ignore the inputs and specify an inner rank of 4 for all params
    tune_vectors=False, # Tell Lorax to put all the biases in the frozen params tree instead of the tunable params tree
    is_leaf=is_leaf
)

# Lorax splits the parameters into two pytrees:
# freeze_params: Anything which received the value lorax.LORA_FREEZE in the spec
# train_params: Pairs of two narrow matrices for values which got positive integers as spec values, or the full parameter if the value lorax.LORA_FULL was in the spec
freeze_params, train_params = lorax.init_lora(quantized_params, lora_spec, jax.random.PRNGKey(1234), is_leaf=is_leaf)

def merge_quantized_with_lora(q_params, lora_freeze):
    return jax.tree_map(
        lambda quant, from_lora: quant if isinstance(quant, jax_gptq.QuantizedMatrix) else from_lora,
        q_params,
        lora_freeze,
        is_leaf=lambda x: isinstance(x, jax_gptq.QuantizedMatrix) # Tell tree_map to treat QuantizedMatrix as a single value instead of a non-leaf node
    )
# Now we put the actual quantized params back
#freeze_params = merge_quantized_with_lora(quantized_params, freeze_params)

The `lorax.lora` transform converts a function from expecting a single pytree in the specified argument to expecting a tuple of two pytrees. It composes with other JAX transforms such as `jax_gptq.use_quantized`, so we can use both at once with no modifications to our model code.

In [ ]:
combined_params = (freeze_params, train_params)

my_model_with_lora_and_quantized_weights = jax_gptq.use_quantized(lorax.lora(my_model))

# The differences from the original `my_model` function are:
# 1. The params argument now expects a tuple of (frozen_params, trainable_params)
# 2. It knows how to compute with quantized weights
quantized_plus_lorax_output = my_model_with_lora_and_quantized_weights(combined_params, quant_data[0])

print(f'GPTQ + Lorax output: {quantized_plus_lorax_output:.3e}')
print(f'GPTQ only: {quant_output:.3e}')

The above values are identical since LoRA initializes one of each pair of matrices as zeros.

Let's look at the size of each pytree:

In [ ]:
count_params = partial(jax.tree_util.tree_reduce,
  lambda acc, param: acc + (param.size if isinstance(param, jnp.ndarray) else 0),
  initializer=0
)

print(f'{count_params(freeze_params):.3e} frozen params')
print(f'{count_params(train_params):.3e} trainable params')

Training with this function is no different from any other JAX function, just make sure to only differentiate your loss with respect to the trainable parameters only. (See the next section for an example).

## GPT-Q-ing + LoRA-ing HuggingFace's Flax GPT-2
I developed these transforms for use with my Haiku models, but since all JAX models are pure functions at the end of the day, it shouldn't matter what framework you use. Lorax supports matmuls and other matmul-like operations such as embedding lookups and 1-D convs.

This is a minimal example of applying the combination to `gpt2-medium`, but it's basically model agnostic.

First let's get the model:

In [ ]:
from transformers import AutoTokenizer, FlaxAutoModelForCausalLM

In [ ]:
model_name = 'gpt2-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model, params = FlaxAutoModelForCausalLM.from_pretrained(model_name, _do_init=False)
params = jax.device_put(params, cpu)

# Because the embedding table is reused as the output linear layer, it'll get quantized at the end of the process, but that will seriously screw up the embedding lookup step, so we'll just save it for later here
orig_embedding_table = np.asarray(params['transformer']['wte']['embedding'])

The GPT-Q paper used real text data for quantization, but for this demo I'll just generate some random values.

In [ ]:
QUANT_BATCH_SIZE = 4
QUANT_EXAMPLE_LENGTH = 64 # I'd recommend making this bigger, but needs to be small to not crash colab

quantization_data = []
key = jax.random.PRNGKey(0)
for _ in range(32):
  batch = jax.random.randint(key, (QUANT_BATCH_SIZE, QUANT_EXAMPLE_LENGTH), 0, 50256)
  quantization_data.append(batch)
  key, = jax.random.split(key, 1)

HuggingFace's models don't have quite the right call signature, so we'll make a wrapper which takes (params, inputs) as an argument:

In [ ]:
def apply_model(params, batch):
  return model(batch, params=params)

quantized_params = jax_gptq.quantize(apply_model, params, quantization_data)

In [ ]:
# Replace the quantized embedding table with the original one
quantized_params['transformer']['wte']['embedding'] = jnp.asarray(orig_embedding_table)
quantized_params = jax.device_put(quantized_params, gpu)

### Finetuning GPT-2 with Lorax

Same as [above](https://colab.research.google.com/drive/18rkULbWqk7mNZDx7Scx-JS3p_s45mgok#scrollTo=HKkhcjx9zJy6&line=3&uniqifier=1), we get the original param structure to tell Lorax how to initialize the LoRA params, then merge the quantized params back in after.

In [ ]:
# Get pre-quantization param tree (some nodes will just be abstract values)
orig_params_or_shapes = jax_gptq.utils.quantized_params_to_shaped_arrays(quantized_params)

# Tell Lorax which leaves should be frozen/fully trained/LoRA trained
spec = lorax.simple_spec(
    orig_params_or_shapes,
    lambda path, arr: 16 if any(pattern in path for pattern in ['c_attn', 'mlp']) else lorax.LORA_FREEZE,
    tune_vectors=True
)

# Initialize parameters
key, init_key = jax.random.split(key)
freeze_params, train_params = lorax.init_lora(
    orig_params_or_shapes,
    spec,
    init_key
)

# Put the quantized params back into the frozen param tree
freeze_params = merge_quantized_with_lora(quantized_params, freeze_params)
combined_params = freeze_params, train_params

Now we can just transform the `apply_model` function and it will use both LoRA and 4-bit quantized parameters

In [ ]:
quantized_plus_lora_fn = jax_gptq.use_quantized(lorax.lora(apply_model))

### Training
Training isn't actually any different from normal training, since you can just think of `freeze_params` as being a constant argument, but here's a demo for completness.

First I'll define a toy corpus which demonstrates Alan's love of cats and Grace's dislike of them.

In [ ]:
CATS = ['lions', 'tigers', 'cheetahs', 'cats', 'ocelots', 'kittens']
DOGS = ['wolves', 'dogs', 'coyotes', 'huskies', 'poodles', 'puppies']

CAT_LOVER = 'Alan'
DOG_LOVER = 'Grace'

dataset = []
for name, polarity in [(CAT_LOVER, True), (DOG_LOVER, False)]:
  liked, disliked = (CATS, DOGS) if polarity else (DOGS, CATS)
  for kind in liked:
    dataset.append(f'{name}: {kind}? I love them!')
    dataset.append(f'{name}: Hey look at those {kind}, that\'s pretty cool')

  for kind in disliked:
    dataset.append(f'{name}: {kind}? I hate them!')
    dataset.append(f'{name}: Oh no, some {kind}! How scary!')

tokenized_data = [jnp.asarray(tokenizer.encode(ex)) for ex in dataset]
max_len = max(ex.shape[0] for ex in tokenized_data)
# Pad the data to speed up jitting. Not worrying about masking due to laziness.
tokenized_data = [jnp.pad(ex, (0, max_len - ex.shape[0])) for ex in tokenized_data]

jitted_model = jax.jit(quantized_plus_lora_fn)


In [ ]:
def make_prediction(params, prefix):
  tokens = jnp.asarray(tokenizer.encode(prefix))
  logits = jitted_model(params, tokens[None]).logits
  
  logprobs = jnp.exp(jax.nn.log_softmax(logits[0, -1]))
  pred_probs, pred_words = jax.lax.top_k(logprobs, 5)

  print(f'Predictions for: "{prefix}"')
  for i, (word_id, prob) in enumerate(zip(pred_words, pred_probs), 1):
    print(f'{i}. {tokenizer.decode([word_id])} - {prob:.2%}')
  print()

test_examples = [
    f'{CAT_LOVER}: jaguars? I',
    f'{DOG_LOVER}: jaguars? I'
]

Let's look at the next word predictions of the unmodified model:

In [ ]:
for ex in test_examples:
  make_prediction(combined_params, ex)

Next we set up a standard training loop. The only difference is that we keep the train/freeze params separate for the optimizer. There's no differences needed for the quantization.

I'll just train with a batch size of 1 here since I don't want to bother with masking, but the transformed model function is fully compatible with vmap etc.

In [ ]:
def loss_fn(train_params, freeze_params, seq):
  inputs = seq[:-1]
  targets = seq[1:]

  combined_params = (freeze_params, train_params)
  logits = quantized_plus_lora_fn(combined_params, inputs[None]).logits[0]
  logprobs = jax.nn.log_softmax(logits)
  losses = -jnp.take_along_axis(logprobs, targets[:, None], axis=-1)
  return jnp.mean(losses)

optimizer = optax.adamw(learning_rate=1e-4, weight_decay=1e-4)
opt_state = optimizer.init(combined_params[1])

@jax.jit
def update_fn(combined_params, opt_state, example):
  freeze_params, train_params = combined_params

  # The main thing is that we have to split up the params here so that JAX knows what to differentiate with respect to
  loss, grads = jax.value_and_grad(loss_fn)(train_params, freeze_params, example)

  updates, opt_state = optimizer.update(grads, opt_state, params=train_params)
  new_train_params = optax.apply_updates(train_params, updates)
  return (freeze_params, new_train_params), opt_state, loss

In [ ]:
bar = trange(50)
for epoch in bar:
  key, = jax.random.split(key, 1)
  permutation = jax.random.permutation(key, jnp.arange(len(dataset)))
  total_loss = 0
  for index in permutation:
    example = tokenized_data[index]
    combined_params, opt_state, loss = update_fn(combined_params, opt_state, example)
    total_loss += loss
  bar.set_description(f'Epoch {epoch} - Loss: {total_loss / len(tokenized_data):.3e}')

The trained LoRA parameters give us a model which predicts that Alan will love jaguars, and Grace will hate them:

In [ ]:
for example in test_examples:
  make_prediction(combined_params, example)
  print()